# GDP by Country

Latest official GDP figures published by the World Bank. Population figures based on United Nations data.
World's GDP is $80,934,771,028,340 (nominal, 2017).

# Import Libraries

In [1]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup

# Fetch Data

In [2]:
url = 'https://www.worldometers.info/gdp/gdp-by-country/'

r = requests.get(url)

print(f"Status Code: {r.status_code} -- Content-Type: {r.headers['Content-Type']} -- Encoding: {r.encoding}")

Status Code: 200 -- Content-Type: text/html; charset=UTF-8 -- Encoding: UTF-8


# Parse HTML Document

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]-->
<!--[if IE 9]> <html lang="en" class="ie9"> <![endif]-->
<!--[if !IE]><!-->
<html lang="en">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   GDP by Country - Worldometer
  </title>
  <meta content="Countries in the world ranked by Gross Domestic Product (GDP). List and ranking of GDP growth, GDP per capita and couuntry share of World's GDP" name="description"/>
  <link href="/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="/favicon/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link href="/favicon/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
  <

# Navigate The Data Structure

In [4]:
tbody = soup.find('tbody')
trs = tbody.find_all('tr')

index = []
country = []
nominal_gdp = []
gdp_abbrev = []
gdp_growth = []
population = []


for td in trs:
    idx = index.append(td.find_all('td')[0].string)
    ctry = country.append(td.find_all('td')[1].find_all('a')[0].string)
    nom_gdp = nominal_gdp.append(td.find_all('td')[2].string) 
    abbrev_gdp = gdp_abbrev.append(td.find_all('td')[3].string)
    gdpg = gdp_growth.append(td.find_all('td')[4].string)
    pop = population.append(td.find_all('td')[5].string)


# Create DataFrame

In [5]:
data = {
    'Index': index,
    'Country': country,
    'Nominal_GDP': nominal_gdp,
    'Abbreviated_GDP': gdp_abbrev,
    'GDP_Growth': gdp_growth,
    'Population': population
}

df = pd.DataFrame(data=data)

# Head and Tail

In [6]:
df

,Index,Country,Nominal_GDP,Abbreviated_GDP,GDP_Growth,Population
0,1,United States,"$19,485,394,000,000",$19.485 trillion,2.27%,"325,084,756"
1,2,China,"$12,237,700,479,375",$12.238 trillion,6.90%,"1,421,021,791"
2,3,Japan,"$4,872,415,104,315",$4.872 trillion,1.71%,"127,502,725"
3,4,Germany,"$3,693,204,332,230",$3.693 trillion,2.22%,"82,658,409"
4,5,India,"$2,650,725,335,364",$2.651 trillion,6.68%,"1,338,676,785"
...,...,...,...,...,...,...
185,186,Micronesia,"$336,427,500",$336 million,3.20%,"532,899"
186,187,Palau,"$289,823,500",$290 million,-3.57%,"17,808"
187,188,Marshall Islands,"$204,173,430",$204 million,3.60%,"58,058"
188,189,Kiribati,"$185,572,502",$186 million,0.33%,"114,158"


In [7]:
df.drop(columns=['Index', 'Nominal_GDP'], inplace=True)                     # drop index and nominal gdp columns

In [8]:
df

,Country,Abbreviated_GDP,GDP_Growth,Population
0,United States,$19.485 trillion,2.27%,"325,084,756"
1,China,$12.238 trillion,6.90%,"1,421,021,791"
2,Japan,$4.872 trillion,1.71%,"127,502,725"
3,Germany,$3.693 trillion,2.22%,"82,658,409"
4,India,$2.651 trillion,6.68%,"1,338,676,785"
...,...,...,...,...
185,Micronesia,$336 million,3.20%,"532,899"
186,Palau,$290 million,-3.57%,"17,808"
187,Marshall Islands,$204 million,3.60%,"58,058"
188,Kiribati,$186 million,0.33%,"114,158"
